# Timing the response time for the web application

In [1]:
import base64
import time
from io import BytesIO
from pathlib import Path

import httpx
import numpy as np

In [2]:
num_attempts = 200

## Timing full-text search

In [3]:
timings = []

with httpx.Client() as client:
    for i in range(num_attempts):
        t0 = time.monotonic_ns()
        client.get("https://dh.nb.no/run/bildesok/search?search=fyrtaarn&embedding_type=CLIP&text_embedding_type=TF-IDF").content
        t1 = time.monotonic_ns()
        timings.append(t1 - t0)

print("mean:", 1e-9 * np.mean(timings), "std:", 1e-9 * np.std(timings))

mean: 0.289032284325 std: 0.11264474133973382


## Timing ID-based search (using per-computed embeddings)

In [4]:
timings = []

with httpx.Client() as client:
    for i in range(num_attempts):
        t0 = time.monotonic_ns()
        client.get("https://dh.nb.no/run/bildesok/find_similar_images?id_=5091139e-4c7f-010f-b42d-d01bf3e069cc&embedding_type=SigLIP").content
        t1 = time.monotonic_ns()
        timings.append(t1 - t0)

print("mean:", 1e-9 * np.mean(timings), "std:", 1e-9 * np.std(timings))

mean: 0.29471211954500004 std: 0.1174948284260663


## Timing image uploads

In [5]:
timings = []
image = Path("../data/analysis/query_examples/1-ivar_aasen.jpg").read_bytes()
image_base64 = base64.b64encode(image).decode('utf-8')

with httpx.Client() as client:
    for i in range(num_attempts):
        t0 = time.monotonic_ns()
        client.post(
            "https://dh.nb.no/run/bildesok/image_upload",
            data={"embedding_type": "SigLIP", "text_embedding_type": "TF-IDF"},
            files={"image": ("ivar_aasen.jpg", BytesIO(image), "image/jpeg")}
        ).content
        t1 = time.monotonic_ns()
        timings.append(t1 - t0)

print("mean:", 1e-9 * np.mean(timings), "std:", 1e-9 * np.std(timings))

mean: 2.7792418706050004 std: 0.25390381988266136
